<a href="https://colab.research.google.com/github/tanishq150802/TrueFoundry_Intern_submission/blob/main/Airline_sentiment_W2V_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
file_path='/content/drive/MyDrive/TrueFoundry/airline_sentiment_analysis.csv'

In [ ]:
import pandas as pd 
class reader(): #class for reading the training data
  def __init__(self):
    self.df=pd.read_csv(file_path)
  def restructure(self):
    res=self.df.drop(['Unnamed: 0'], axis=1)
    columns_titles = ["text","airline_sentiment"]
    res=res.reindex(columns=columns_titles)
    return res

In [ ]:
read=reader()
df=read.restructure()
df.head()

,text,airline_sentiment
0,@VirginAmerica plus you've added commercials t...,positive
1,@VirginAmerica it's really aggressive to blast...,negative
2,@VirginAmerica and it's a really big bad thing...,negative
3,@VirginAmerica seriously would pay $30 a fligh...,negative
4,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [ ]:
df['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
from sklearn.utils import resample,shuffle
df_1 = df[df['airline_sentiment'] == 'positive']
other_df = df[df['airline_sentiment'] == 'negative']
df_1_upsampled = resample(df_1,n_samples=10000,random_state=40)
df = pd.concat([df_1_upsampled,other_df])
print(df['airline_sentiment'].value_counts())
df.head(3)

positive    10000
negative     9178
Name: airline_sentiment, dtype: int64


,text,airline_sentiment
10640,@AmericanAir thanks. They did not charge anyth...,positive
9348,@AmericanAir I was happy to purchase the upgra...,positive
10219,@AmericanAir SFO. Natt (the agent who helped m...,positive


In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() #binary label 1: positive, 0: negative
df['airline_sentiment']= label_encoder.fit_transform(df['airline_sentiment'])
df.head(3)

,text,airline_sentiment
10640,@AmericanAir thanks. They did not charge anyth...,1
9348,@AmericanAir I was happy to purchase the upgra...,1
10219,@AmericanAir SFO. Natt (the agent who helped m...,1


In [ ]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [ ]:
import re
import string
import nltk

def cleaning_mentions(text):
  return re.sub("@[A-Za-z0-9_]+","", text)
df['text'] = df['text'].apply(lambda x: cleaning_mentions(x))

def cleaning_non_alpha(text):
  return re.sub("[^a-z0-9]"," ", text)
df['text'] = df['text'].apply(lambda x: cleaning_non_alpha(x))

STOPWORDS = set(stopwordlist) #cleaning stopwords
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df['text'] = df['text'].apply(lambda text: cleaning_stopwords(text))

def cleaning_URLs(data): #cleaning URLs (if any)
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ', data)
df['text'] = df['text'].apply(lambda x: cleaning_URLs(x))

english_punctuations = string.punctuation #removing punctuations
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['text'] = df['text'].apply(lambda x: cleaning_punctuations(x))

def cleaning_repeating_char(text): #removing repeating characters
    return re.sub(r'(.)1+', r'1', text)
df['text'] = df['text'].apply(lambda x: cleaning_repeating_char(x))

def cleaning_numbers(data): #removing numbers
    return re.sub('[0-9]+', '', data)
df['text'] = df['text'].apply(lambda x: cleaning_numbers(x))

st = nltk.PorterStemmer() #stemming is generally more suitable for sentiment analysis problems
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
df['text'] = df['text'].apply(lambda x: stemming_on_text(x))

# from nltk.stem import WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()
# def lemmatizer(text):
#   lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
#   return lemm_text
# df['text'] = df['text'].apply(lambda x:lemmatizer(x))

df.head(3)

,text,airline_sentiment
10640,thanks hey not charge anything end good,1
9348,happy purchase upgrade f avail next flight,1
10219,att agent helped really awesome job,1


In [ ]:
df=df.sample(frac=1)
df.head()

,text,airline_sentiment
3494,agree ne airline fly away favorite usiness ht...,1
10856,idn really need anything aw mentions often neg...,1
11377,tell luggage stayed inside plane ook video htt...,0
5033,bout time time next flight hursday,1
3892,hanks nly minor delay not big deal ppreciate ...,1


In [ ]:
#defining & calculating maxlen for pad sequencing
maxlen=0
for i in df.index:
  t=0
  for id in df['text'][i]:
    if(id==' '):
      t=t+1
  maxlen=max(maxlen,t+1)
print(maxlen)

24


In [ ]:
import gensim
txt=[nltk.word_tokenize(t) for t in df['text'].values]
txt

[['agree',
  'ne',
  'airline',
  'fly',
  'away',
  'favorite',
  'usiness',
  'http',
  'co',
  'o',
  'la',
  'n'],
 ['idn',
  'really',
  'need',
  'anything',
  'aw',
  'mentions',
  'often',
  'negative',
  'eep',
  'good',
  'work',
  'gratitude'],
 ['tell',
  'luggage',
  'stayed',
  'inside',
  'plane',
  'ook',
  'video',
  'http',
  'co',
  'okk',
  'c'],
 ['bout', 'time', 'time', 'next', 'flight', 'hursday'],
 ['hanks',
  'nly',
  'minor',
  'delay',
  'not',
  'big',
  'deal',
  'ppreciate',
  'concern',
  'though',
  'oarding',
  'ou',
  'amazing',
  'service'],
 ['ticket',
  'booked',
  'miles',
  'orry',
  'sold',
  'ticket',
  'company',
  'merged',
  'but',
  'fix'],
 ['thanks', 'filled', 'ow', 'luck', 'common'],
 ['hold', 'hour', 'need', 'rebook', 'flight', 'ancelled', 'lighted', 'help'],
 ['hanks', 'making', 'good', 'ancelled', 'lightled', 'flight'],
 ['thank',
  'reply',
  'fill',
  'form',
  'submit',
  'requested',
  'ood',
  'know',
  'paying',
  'attention'],
 

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
w2v=Word2Vec(txt, min_count=2, size=100)

In [ ]:
w2v.most_similar("bad")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('awful', 0.9988214373588562),
 ('disappointed', 0.998690128326416),
 ('lack', 0.9983086585998535),
 ('headed', 0.9982846975326538),
 ('gets', 0.9982776045799255),
 ('irways', 0.9982577562332153),
 ('usually', 0.9982377290725708),
 ('web', 0.9982335567474365),
 ('usairwaysfail', 0.9982233047485352),
 ('ude', 0.9982213973999023)]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.1, random_state=37)
X_train[0]

0    plus added commercials experience tacky
0    plus added commercials experience tacky
Name: text, dtype: object

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index
vocab_size = len(word_index)+1
print(vocab_size)

10467


In [ ]:
X_learn = tokenizer.texts_to_sequences(X_train)
X_eval = tokenizer.texts_to_sequences(X_test)
len(X_learn)

17260

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_learn, padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_eval, padding='post', maxlen=maxlen)

In [ ]:
y_tr=y_train.to_numpy()
y_ts=y_test.to_numpy()
y_tr = y_tr.reshape(-1,1)
y_ts = y_ts.reshape(-1,1)
y_tr

array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
W2V_SIZE=100
import numpy as np
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
  if word in w2v:
    embedding_matrix[i] = w2v[word]
print(embedding_matrix.shape)

(10467, 100)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print(X_train_pad.shape, y_tr.shape)

(17260, 24) (17260, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, SpatialDropout1D, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
# from keras.optimizers import SGD
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]
class classifier(): #model and training class
  def __init__(self,model):
    self.model=model
  def add_layers(self,model):
    self.model.add(Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], input_length=maxlen))
    self.model.add(SpatialDropout1D(0.4))
    self.model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
    self.model.add(Dense(1,activation='sigmoid'))
    return self.model
  def compiler(self,model):
    self.model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
  def summary(self,model):
    print(self.model.summary())
  def fit(self,model,x,y,epoch, step, batch_size=32, verbose='auto', callbacks=callbacks, validation_split=0.1):
    self.model.fit(x,y,epochs=epoch, steps_per_epoch=step, batch_size=batch_size, verbose=verbose,callbacks=callbacks, validation_split=validation_split)

In [ ]:
model=Sequential()
obj=classifier(model)
model=obj.add_layers(model)
obj.compiler(model)
obj.summary(model)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 24, 100)           1046700   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 24, 100)          0         
 lDropout1D)                                                     
                                                                 
 lstm_12 (LSTM)              (None, 176)               195008    
                                                                 
 dense_4 (Dense)             (None, 1)                 177       
                                                                 
Total params: 1,241,885
Trainable params: 1,241,885
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
obj.fit(model,X_train_pad, y_train, 5, 125)

Epoch 1/5
125/125 [==============================] - ETA: 0s - loss: 0.0878 - accuracy: 0.9685

125/125 [==============================] - 14s 111ms/step - loss: 0.0878 - accuracy: 0.9685 - val_loss: 0.1325 - val_accuracy: 0.9565 - lr: 0.0010
Epoch 2/5
125/125 [==============================] - ETA: 0s - loss: 0.0949 - accuracy: 0.9672

125/125 [==============================] - 14s 109ms/step - loss: 0.0949 - accuracy: 0.9672 - val_loss: 0.1204 - val_accuracy: 0.9635 - lr: 0.0010
Epoch 3/5
125/125 [==============================] - ETA: 0s - loss: 0.0968 - accuracy: 0.9725

125/125 [==============================] - 14s 108ms/step - loss: 0.0968 - accuracy: 0.9725 - val_loss: 0.1053 - val_accuracy: 0.9676 - lr: 0.0010
Epoch 4/5
125/125 [==============================] - ETA: 0s - loss: 0.0828 - accuracy: 0.9729

125/125 [==============================] - 14s 109ms/step - loss: 0.0828 - accuracy: 0.9729 - val_loss: 0.1065 - val_accuracy: 0.9687 - lr: 0.0010
Epoch 5/5
125/125 [==============================] - ETA: 0s - loss: 0.0589 - accuracy: 0.9815

125/125 [==============================] - 14s 110ms/step - loss: 0.0589 - accuracy: 0.9815 - val_loss: 0.1125 - val_accuracy: 0.9647 - lr: 0.0010


In [ ]:
# from sklearn.metrics import accuracy_score
# y_sol=model.predict(X_test_pad)
# y_pred=np.argmax(y_sol,axis=1)
# print(accuracy_score(y_ts, y_pred))

score = model.evaluate(X_test_pad, y_ts, batch_size=32)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

60/60 [==============================] - 1s 9ms/step - loss: 0.1044 - accuracy: 0.9687

ACCURACY: 0.9687173962593079
LOSS: 0.10440192371606827


In [ ]:
from sklearn.metrics import accuracy_score
y_sol=model.predict(X_test_pad)
y_p=y_sol
for i in y_sol:
  if(i[0]>=0.5):
    i[0]=1
  else:
    i[0]=0
y_sol[:10]

60/60 [==============================] - 1s 8ms/step


array([[0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.]], dtype=float32)

In [ ]:
print(accuracy_score(y_ts, y_sol))

0.9687174139728885


In [ ]:
#Inference class:

class inference():
  def __init__(self,exp):
    self.exp=exp
  def preprocess(self,exp):
    exp=cleaning_mentions(exp)
    exp=cleaning_non_alpha(exp)
    exp=cleaning_numbers(exp)
    exp=cleaning_URLs(exp)
    exp=cleaning_punctuations(exp)
    exp=cleaning_repeating_char(exp)
    exp=cleaning_numbers(exp)
    exp=stemming_on_text(exp)
    exp = tokenizer.texts_to_sequences([exp])
    exp = pad_sequences(exp, padding='post', maxlen=maxlen)
    return exp
  def pred(self,exp):
    pred=model.predict(exp)
    if(pred[0][0]>=0.5):
      return ["Positive",pred]
    else:
      return ["Negative",pred]

In [ ]:
exp="good"
obj2=inference(exp)
exp=obj2.preprocess(exp)
print(obj2.pred(exp))

1/1 [==============================] - 0s 85ms/step
['Positive', array([[0.93742764]], dtype=float32)]


In [ ]:
y_p

array([[0.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

In [ ]:
model.save('tf_better.h5')

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)